In [12]:
import pandas as pd
import nltk
import string
import re
import demoji



from time import sleep
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer





In [13]:

demoji.download_codes()


stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')


C:\Users\USER\AppData\Local\Temp\ipykernel_9428\183877869.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [18]:
 def getComments(url):

    # Set up Chrome driver options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    # Initialize ChromeDriver with an implicit wait of 20 seconds
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(50)

    # Open the video page
    driver.get(url)
    sleep(7)

    # Wait for the comment section to load
    wait = WebDriverWait(driver, 40)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="continuations"]')))

    scroll_pause_time = 10
    scroll_count = 0
    comment_list = []

    while len(comment_list) < 50 and scroll_count < 10:
        # Scroll down to the bottom of the page to load more comments
        driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight);')
        scroll_count += 1
        sleep(scroll_pause_time)

        # Find the comment div element and extract the comments
        comment_div = driver.find_element(By.XPATH, '//*[@id="contents"]')
        comments = comment_div.find_elements(By.XPATH, '//*[@id="content-text"]')
        for comment in comments:
            comment_list.append(comment.text.strip())
            
    # print(f"Scraped {len(comment_list)} comments")

    driver.quit()
    return comment_list


In [19]:
urls = ["https://www.youtube.com/watch?v=zz70o2Ia4X0", "https://www.youtube.com/watch?v=XUkV3RsjGrs","https://www.youtube.com/watch?v=UORoRpW8tnA" ,
        "https://www.youtube.com/watch?v=CM7TbmdOeAs", "https://youtu.be/zhoTX0RRXPQ", "https://youtu.be/BZPV3Cf3pLo","https://youtu.be/nlowwv-I81M" ]

In [20]:
comments = []
for url in tqdm(urls):
    com = getComments(url)
    comments.extend(com)
    sleep(10)
    

100%|██████████| 7/7 [06:24<00:00, 54.97s/it]


In [21]:
df = pd.DataFrame({'comments':comments})

In [22]:
df.to_csv('comments.csv', index = False)

<module 'emoji' from 'c:\\Users\\USER\\Google Drive\\PROJECTS PERSONAL\\Data Science projects\\Sentiment_analyis\\Tech_review_sentiment\\.conda\\Lib\\site-packages\\emoji\\__init__.py'>

In [23]:


def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers and special characters
    text = re.sub('[^a-zA-Z ]', '', text)

    # Remove URLs and mentions
    text = re.sub(r'http\S+', '', text)
    text = re.sub('@[^\s]+', '', text)

    # Remove emojis
    text = remove_emojis(text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stop words
    words = [word for word in words if word not in stop_words]

    # Stem the words
    words = [stemmer.stem(word) for word in words]

    # Join the words back into a string
    text = ' '.join(words)

    return text

def remove_emojis(text):
    return demoji.replace(text, '')


In [24]:
df.comments.apply(clean_text)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\USER/nltk_data'
    - 'c:\\Users\\USER\\Google Drive\\PROJECTS PERSONAL\\Data Science projects\\Sentiment_analyis\\Tech_review_sentiment\\.conda\\nltk_data'
    - 'c:\\Users\\USER\\Google Drive\\PROJECTS PERSONAL\\Data Science projects\\Sentiment_analyis\\Tech_review_sentiment\\.conda\\share\\nltk_data'
    - 'c:\\Users\\USER\\Google Drive\\PROJECTS PERSONAL\\Data Science projects\\Sentiment_analyis\\Tech_review_sentiment\\.conda\\lib\\nltk_data'
    - 'C:\\Users\\USER\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
***************************

In [29]:
clean_comments = []
for comment in comments:
    clean_comment = clean_text(comment)
    clean_comments.append(clean_comment)

df['clean_comments'] = clean_comments



AttributeError: module 'emoji' has no attribute 'get_emoji_regexp'